In [1]:
import openai
import json
from sklearn.metrics import classification_report
import pandas as pd
from tqdm import tqdm
from credentials import OPENAPI_KEY
openai.api_key = OPENAPI_KEY

In [3]:
train_ds_path = "DS_PATH/train.jsonl"
openai.File.create(file=open(train_ds_path, "rb"), purpose="fine-tune")

<File file id=file-BNsP6BVTXWCIeh8hx51lKN28 at 0x14654908fc20> JSON: {
  "object": "file",
  "id": "file-BNsP6BVTXWCIeh8hx51lKN28",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 3316604,
  "created_at": 1714154891,
  "status": "processed",
  "status_details": null
}

In [ ]:
openai.FineTuningJob.create(training_file="FILE_ID", model="gpt-3.5-turbo")

In [ ]:
openai.FineTuningJob.retrieve("FINETUNE_ID")

In [3]:
test_ds_path = "DS_PATH/test.jsonl"
with open(test_ds_path, "r") as f:
    test_dataset = [json.loads(line) for line in f]

In [4]:
test_dataset[0]["messages"]

[{'role': 'system',
  'content': 'VRL-Detector is a chatbot that can detect if the given Turkish text contains Hate Speech.'},
 {'role': 'user',
  'content': 'Does this Turkish text contain hate speech? : "#Yıkılasınisrail https://t.co/Q002Qk4vWn" '},
 {'role': 'assistant', 'content': 'yes'}]

In [8]:
preds = []
refs = []
for example in tqdm(test_dataset):
    completion = openai.ChatCompletion.create(
        model="MODEL_ID",
        messages=example["messages"][:2]
        )
    model_output = completion.choices[0].message
    ground_truth = example["messages"][-1]["content"]
    preds.append(model_output["content"])
    refs.append(ground_truth)

100%|██████████| 8805/8805 [57:23<00:00,  2.56it/s]  


In [18]:

output_df = pd.DataFrame({"preds": preds, "refs": refs})
output_df.to_csv("OUTPUT_PATH/output.csv", index=False)

In [14]:
output_labels = ["no", "yes"]
for index in range(len(preds)):
    if preds[index] not in output_labels:
        preds[index] = "no" 

In [15]:
print(classification_report(refs, preds, digits=4))

              precision    recall  f1-score   support

          no     0.8561    0.8258    0.8407      6121
         yes     0.6324    0.6833    0.6569      2684

    accuracy                         0.7824      8805
   macro avg     0.7442    0.7546    0.7488      8805
weighted avg     0.7879    0.7824    0.7847      8805



In [11]:
set(preds)

{'no', 'yes'}

In [136]:
temp_df  = pd.read_csv("OUTPUT_PATH/output.csv")

temp_df.head()

,preds,refs
0,positive,neutral
1,neutral,neutral
2,neutral,neutral
3,neutral,neutral
4,neutral,negative
